# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

def nhl_correlation(): 
    nhl_df=pd.read_csv("assets/nhl.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]

    nhl_df.columns = [x.lower().strip() for x in nhl_df.columns]
    nhl_df = nhl_df[nhl_df['year'] == 2018]
    cities.columns = [x.lower().strip() for x in cities.columns]
    cities.rename(columns = {"population (2016 est.)[8]": "population",
                            "metropolitan area": "city"}, inplace=True)
    
    for key,value in cities.iteritems():
        value = value.replace(r"-?[ ]?\[(.*?)\]", "",regex=True, inplace=True)

    vals_to_replace = {'—':np.nan, "":np.nan}
    for key,value in cities.iteritems():
        value = value.replace(vals_to_replace, inplace=True)
        
    cities_nhl = cities[['city', 'population', 'nhl']]
    cities_nhl = cities_nhl[cities_nhl['nhl'].notna()]
    cities_nhl.index = pd.RangeIndex(len(cities_nhl.index)) # This is faster than reset_index
    
    nhl = nhl_df[['team', 'w', 'l']]
    
    nhl = nhl.replace(r"\*", "",regex=True)
    nhl.drop(nhl.index[[0,9,18,26]], inplace=True)
    nhl.index = pd.RangeIndex(len(nhl.index))

    nhl["team_only"]=nhl['team'].apply(lambda x: x.rsplit(" ")[-1])
    nhl.loc[2, 'team_only'] = 'Maple Leafs'
    nhl.loc[4, 'team_only'] = 'Red Wings'
    nhl.loc[11, 'team_only'] = 'Blue Jackets'
    nhl.loc[23, 'team_only'] = 'Golden Knights'

    # Some cities have multiple teams and joined in one
    nhl.loc[15, 'team_only'] = 'RangersIslandersDevils'
    nhl.loc[14, 'team_only'] = 'RangersIslandersDevils'
    nhl.loc[12, 'team_only'] = 'RangersIslandersDevils'
    nhl.loc[26, 'team_only'] = 'KingsDucks'
    nhl.loc[24, 'team_only'] = 'KingsDucks'

    nhl[['w', 'l']] = nhl[['w', 'l']].astype(float)

    nhl = nhl.groupby(by='team_only')['w','l'].mean()
    nhl.reset_index(inplace=True)
    
    data = pd.merge(cities_nhl,nhl, how='inner',left_on='nhl', right_on='team_only')
    data['ratio'] = data['w']/(data['w']+data['l'])
    data['population'] = data['population'].astype(float)

#     raise NotImplementedError()
    
    population_by_region = list(data['population']) # pass in metropolitan area population from cities
    win_loss_by_region = list(data['ratio']) # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    
    return (stats.pearsonr(population_by_region, win_loss_by_region))[0]

In [2]:
nhl_correlation()

0.012308996455744285

## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [9]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

def nba_correlation():
    
    nba_df=pd.read_csv("assets/nba.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    
    nba_df.columns = [x.lower().strip() for x in nba_df.columns]
    nba_df = nba_df[nba_df['year'] == 2018]
    cities.columns = [x.lower().strip() for x in cities.columns]
    cities.rename(columns = {"population (2016 est.)[8]": "population",
                            "metropolitan area": "city"}, inplace=True)
    
    for key,value in cities.iteritems():
        value = value.replace(r"-?[ ]?\[(.*?)\]", "",regex=True, inplace=True)

    vals_to_replace = {'—':np.nan, "":np.nan}
    for key,value in cities.iteritems():
        value = value.replace(vals_to_replace, inplace=True)
        
    cities_nba = cities[['city', 'population', 'nba']]
    cities_nba = cities_nba[cities_nba['nba'].notna()]
    cities_nba.index = pd.RangeIndex(len(cities_nba.index)) # This is faster than reset_index
    
    nba_df = nba_df[['team', 'w', 'l']]
    nba_df['team'] = nba_df['team'].replace(r"\((.*?)\)", "",regex=True)
    nba_df['team'] = nba_df['team'].replace(r"(\*\s+)", "",regex=True)

    nba_df["team_only"]=nba_df['team'].apply(lambda x: x.rsplit(" ")[-1])
    nba_df["team_only"] = nba_df["team_only"].replace(r"\s+", "",regex=True)

    # # Some cities have multiple teams and joined in one
    nba_df.loc[24, 'team_only'] = 'LakersClippers'
    nba_df.loc[25, 'team_only'] = 'LakersClippers'
    nba_df.loc[17, 'team_only'] = 'Trail Blazers'
    nba_df.loc[10, 'team_only'] = 'KnicksNets'
    nba_df.loc[11, 'team_only'] = 'KnicksNets'

    nba_df[['w', 'l']] = nba_df[['w', 'l']].astype(float)

    nba_df = nba_df.groupby(by='team_only')['w','l'].mean()
    nba_df.reset_index(inplace=True)
    
    data = pd.merge(cities_nba,nba_df, how='inner',left_on='nba', right_on='team_only')
    data['ratio'] = data['w']/(data['w']+data['l'])
    data['population'] = data['population'].astype(float)
#     raise NotImplementedError()
    
    population_by_region = list(data['population']) # pass in metropolitan area population from cities
    win_loss_by_region = list(data['ratio']) # pass in win/loss ratio from nba_df in the same order as cities["Metropolitan

    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"

    return (stats.pearsonr(population_by_region, win_loss_by_region))[0]

In [10]:
nba_correlation()

-0.1765716025284462

## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [5]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

def mlb_correlation(): 
    mlb_df=pd.read_csv("assets/mlb.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    cities.columns = [x.lower().strip() for x in cities.columns]
    cities.rename(columns = {"population (2016 est.)[8]": "population",
                            "metropolitan area": "city"}, inplace=True)

    mlb_df.columns = [x.lower().strip() for x in mlb_df.columns]
    mlb_df = mlb_df[mlb_df['year'] == 2018]
    
    for key,value in cities.iteritems():
        value = value.replace(r"-?[ ]?\[(.*?)\]", "",regex=True, inplace=True)
    
    vals_to_replace = {'—':np.nan, "":np.nan}
    for key,value in cities.iteritems():
        value = value.replace(vals_to_replace, inplace=True)
    cities_mlb = cities[['city', 'population', 'mlb']]
    cities_mlb = cities_mlb[cities_mlb['mlb'].notna()]
    cities_mlb.index = pd.RangeIndex(len(cities_mlb.index)) # This is faster than reset_index
    
    mlb_df = mlb_df[['team', 'w', 'l']]
    mlb_df["team_only"]=mlb_df['team'].apply(lambda x: x.rsplit(" ")[-1])

    # # Some cities have multiple teams and joined in one
    mlb_df.loc[3, 'team_only'] = 'Blue Jays'
    mlb_df.loc[0, 'team_only'] = 'Red Sox'
    mlb_df.loc[21, 'team_only'] = 'CubsWhite Sox'
    mlb_df.loc[8, 'team_only'] = 'CubsWhite Sox'

    mlb_df.loc[1, 'team_only'] = 'YankeesMets'
    mlb_df.loc[18, 'team_only'] = 'YankeesMets'
    mlb_df.loc[28, 'team_only'] = 'GiantsAthletics'
    mlb_df.loc[11, 'team_only'] = 'GiantsAthletics'
    mlb_df.loc[13, 'team_only'] = 'DodgersAngels'
    mlb_df.loc[25, 'team_only'] = 'DodgersAngels'


    mlb_df[['w', 'l']] = mlb_df[['w', 'l']].astype(float)

    mlb_df = mlb_df.groupby(by='team_only')['w','l'].mean()
    mlb_df.reset_index(inplace=True)
    
    data = pd.merge(cities_mlb,mlb_df, how='inner',left_on='mlb', right_on='team_only')
    data['ratio'] = data['w']/(data['w']+data['l'])
    data['population'] = data['population'].astype(float)
    
#     raise NotImplementedError()
    
    population_by_region = list(data['population']) # pass in metropolitan area population from cities
    win_loss_by_region = list(data['ratio']) # pass in win/loss ratio from mlb_df in the same order as cities["Metropolitan

    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"

    return (stats.pearsonr(population_by_region, win_loss_by_region))[0]

In [6]:
mlb_correlation()

0.1505230448710485

## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [12]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

def nfl_correlation(): 
    
    nfl_df=pd.read_csv("assets/nfl.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    cities.columns = [x.lower().strip() for x in cities.columns]
    cities.rename(columns = {"population (2016 est.)[8]": "population",
                            "metropolitan area": "city"}, inplace=True)
    
    nfl_df.columns = [x.lower().strip() for x in nfl_df.columns]
    nfl_df = nfl_df[nfl_df['year'] == 2018]
    
    for key,value in cities.iteritems():
        value = value.replace(r"-?[ ]?\[(.*?)\]", "",regex=True, inplace=True)
    
    vals_to_replace = {'—':np.nan, "":np.nan}
    for key,value in cities.iteritems():
        value = value.replace(vals_to_replace, inplace=True)
        
    cities_nfl = cities[['city', 'population', 'nfl']]
    cities_nfl = cities_nfl[cities_nfl['nfl'].notna()]
    cities_nfl.index = pd.RangeIndex(len(cities_nfl.index)) # This is faster than reset_index
    
    nfl_df = nfl_df[['team', 'w', 'l']]
    nfl_df= nfl_df.replace(r"\*|\+", "",regex=True)
    nfl_df.drop(nfl_df.index[[0,5,10,15,20,25,30,35]], inplace=True)
    nfl_df.index = pd.RangeIndex(len(nfl_df.index))

    nfl_df["team_only"]=nfl_df['team'].apply(lambda x: x.rsplit(" ")[-1])

    nfl_df.loc[3, 'team_only'] = 'GiantsJets'
    nfl_df.loc[19, 'team_only'] = 'GiantsJets'
    nfl_df.loc[13, 'team_only'] = 'RamsChargers'
    nfl_df.loc[28, 'team_only'] = 'RamsChargers'
    nfl_df.loc[15, 'team_only'] = '49ersRaiders'
    nfl_df.loc[30, 'team_only'] = '49ersRaiders'

    nfl_df[['w', 'l']] = nfl_df[['w', 'l']].astype(float)

    nfl_df = nfl_df.groupby(by='team_only')['w','l'].mean()
    nfl_df.reset_index(inplace=True)
    
    data = pd.merge(cities_nfl,nfl_df, how='inner',left_on='nfl', right_on='team_only')
    data['ratio'] = data['w']/(data['w']+data['l'])
    data['population'] = data['population'].astype(float)
    
#     raise NotImplementedError()
    
    population_by_region = list(data['population']) # pass in metropolitan area population from cities
    win_loss_by_region = list(data['ratio']) # pass in win/loss ratio from nfl_df in the same order as cities["Metropolitan

    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"

    return (stats.pearsonr(population_by_region, win_loss_by_region))[0]

In [13]:
nfl_correlation()

0.004922112149349386

## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
nhl_df=pd.read_csv("assets/nhl.csv")
nba_df=pd.read_csv("assets/nba.csv")
nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def sports_team_performance():
    # YOUR CODE HERE
    raise NotImplementedError()
    
    # Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"] and
    # df.loc["NFL","NFL"] should return np.nan
    sports = ['NFL', 'NBA', 'NHL', 'MLB']
    p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)
    
    assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NHL p-value should be around 0.02"
    assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
    return p_values